In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dense, Flatten, BatchNormalization, LeakyReLU, ReLU, Reshape, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import backend
# from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import os

In [2]:
tf.enable_eager_execution()
tf.executing_eagerly()

True

In [3]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
          tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
saved_model = tf.keras.models.load_model("gan_evaluation/gan_evaluation_model.h5")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [5]:
latent_shape = 100
batch_size = 32
n_epochs=50

In [6]:
(x_train_full, y_train_full), (x_test_full, y_test_full) = tf.keras.datasets.mnist.load_data()

# Find indices of labels 5 to 9
train_index = np.squeeze(np.argwhere(y_train_full>=5))

(x_train, y_train) = (x_train_full[train_index], y_train_full[train_index])

x_train_n = np.expand_dims(x_train/255, axis=-1).astype('float32')

num_classes = 5

# convert class vectors to binary class matrices
y_train_b = to_categorical(y_train-5, num_classes)

dataset = tf.data.Dataset.from_tensor_slices(x_train_n)
dataset = dataset.shuffle(buffer_size=100).batch(batch_size, drop_remainder=True).prefetch(1)

In [7]:
# nb of steps
x_train_n.shape[0]/batch_size*n_epochs

45943.75

In [8]:
init = RandomNormal(stddev=0.02)
clip_value = 0.01

In [9]:
critic = tf.keras.Sequential(name="critic")

critic.add(Conv2D(64, (4, 4), strides=(2, 2), padding="same", kernel_initializer=init, input_shape=(28, 28, 1)))
critic.add(BatchNormalization())
critic.add(Dropout(0.4))
critic.add(LeakyReLU(alpha=0.2))

critic.add(Conv2D(64, (4, 4), strides=(2, 2), padding="same", kernel_initializer=init))
critic.add(BatchNormalization())
critic.add(Dropout(0.4))
critic.add(LeakyReLU(alpha=0.2))

critic.add(Flatten())
critic.add(Dense(1, activation="sigmoid"))

In [10]:
critic.summary()

Model: "critic"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 64)        1088      
_________________________________________________________________
batch_normalization (BatchNo (None, 14, 14, 64)        256       
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 64)          65600     
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 7, 64)          256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 64)          0    

In [11]:
generator = tf.keras.Sequential(name="generator")

generator.add(Dense(128*7*7, input_shape=[latent_shape]))
generator.add(LeakyReLU(alpha=0.2))
generator.add(Reshape((7, 7, 128)))

generator.add(Conv2DTranspose(128, (4, 4),strides=(2, 2), padding="same", kernel_initializer=init))
generator.add(BatchNormalization())
generator.add(LeakyReLU(alpha=0.2))

generator.add(Conv2DTranspose(128, (4, 4),strides=(2, 2), padding="same", kernel_initializer=init))
generator.add(BatchNormalization())
generator.add(LeakyReLU(alpha=0.2))

generator.add(Conv2DTranspose(1, (7, 7), padding="same", kernel_initializer=init, activation="tanh"))


In [12]:
generator.summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6272)              633472    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 6272)              0         
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 14, 14, 128)       262272    
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 128)       512       
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 28, 28, 128)       26

In [13]:
gan = tf.keras.Sequential([generator, critic], name="gan")
gan.summary()

Model: "gan"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generator (Sequential)       (None, 28, 28, 1)         1165313   
_________________________________________________________________
critic (Sequential)          (None, 1)                 70337     
Total params: 1,235,650
Trainable params: 1,234,882
Non-trainable params: 768
_________________________________________________________________


In [14]:
def wasserstein_loss(y_true, y_pred):
    return backend.mean(y_true * y_pred)

In [15]:
critic.compile(loss=wasserstein_loss, optimizer=RMSprop(lr=0.0005, momentum=0.9, clipvalue=clip_value), metrics=['accuracy'])
critic.trainable = False
gan.compile(loss=wasserstein_loss, optimizer=RMSprop(lr=0.0005, momentum=0.9, clipvalue=clip_value), metrics=['accuracy'])

In [16]:
def train_gan(gan, dataset, batch_size, latent_shape, n_epochs=n_epochs, n_critic=5):
    c_hist, g_hist = list(), list()
    generator, critic = gan.layers
    for epoch in range(n_epochs):
        for X_batch in dataset:
            
            # phase 1 - training the critic
            c_tmp = list()
            for _ in range(n_critic):
                noise = tf.random.normal(shape=[batch_size, latent_shape])
                generated_images = generator(noise)
#                 print("Generated images shape for critic: {}".format(generated_images.shape))
                X_fake_and_real = tf.concat([generated_images, X_batch], axis=0)
#                 print("Fake and Real images shape: {}".format(X_fake_and_real.shape))
                y1 = tf.constant([[1.]] * batch_size + [[-1.]] * batch_size)
#                 print("Labels for critic shape: {}".format(y1.shape))
                critic.trainable = True
                c_loss, _ = critic.train_on_batch(X_fake_and_real, y1)
                c_tmp.append(c_loss)
                                    
            c_hist.append(np.mean(c_tmp))
            
            # phase 2 - training the generator
            noise = tf.random.normal(shape=[batch_size, latent_shape])
#             print("Generated images shape for generator: {}".format(noise.shape))
            y2 = tf.constant([[-1.]] * batch_size)
#             print("Labels for generator shape: {}".format(y2.shape))
            critic.trainable = False
            g_loss, _ = gan.train_on_batch(noise, y2)
            g_hist.append(g_loss)
        
        print('>%d, c=%.3f, g=%.3f' % (epoch, c_hist[-1], g_loss))
        
        # predictions
        y_prob = saved_model.predict(generated_images)
        for i in range(batch_size):
            label = np.argmax(y_prob[i,:])+5
            prob = np.max(y_prob[i,:])
            # PLOT
            plt.subplot(4, 8, 1 + i)
            plt.axis('off')
            plt.imshow(generated_images[i, :, :, 0], cmap='gray_r')
            plt.title("L={}\nP={:.2f}".format(label, prob), size=10)
            # save plot to file
        filename1 = 'eval_gan_plot_%04d.png' % (epoch+1)
        plt.savefig(os.path.join("gan_evaluation", filename1), dpi=200, size=(10, 10))
        plt.close()
        # save the generator model
        filename2 = os.path.join("gan_evaluation",'attack_gan_model_%04d.h5' % (epoch+1))
        gan.save(filename2)
        # save the critic model
        filename3 = os.path.join("gan_evaluation",'attack_critic_model_weights_%04d.h5' % (epoch+1))
        critic.save_weights(filename3)
        

In [ ]:
train_gan(gan, dataset, batch_size, latent_shape)

>0, c=-0.500, g=-0.947
>1, c=-0.500, g=-0.982
>2, c=-0.500, g=-0.998
>3, c=-0.500, g=-1.000
>4, c=-0.500, g=-0.994
>5, c=-0.500, g=-0.999
>6, c=-0.500, g=-1.000
>7, c=-0.500, g=-0.998
>8, c=-0.500, g=-0.999
>9, c=-0.500, g=-0.978
>10, c=-0.500, g=-0.984
>11, c=-0.500, g=-0.979
>12, c=-0.500, g=-0.986
>13, c=-0.500, g=-0.995
>14, c=-0.500, g=-0.992
>15, c=-0.500, g=-0.990
>16, c=-0.500, g=-0.994
>17, c=-0.500, g=-0.983
